In [1]:
import os

DATA_DIR = "StanfordDroneDataset"
directories = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
print(directories)

['little', 'gates', 'coupa', 'hyang', 'deathCircle', 'quad', 'nexus', 'bookstore']


In [35]:
sample_ann = f"{os.path.join(DATA_DIR, directories[0])}/video0/annotations.txt"
import numpy as np

data = np.genfromtxt(sample_ann, delimiter=' ', dtype="str", encoding='utf-8')
data

array([['0', '818', '1409', ..., '0', '0', '"Pedestrian"'],
       ['0', '818', '1409', ..., '0', '1', '"Pedestrian"'],
       ['0', '818', '1409', ..., '0', '1', '"Pedestrian"'],
       ...,
       ['60', '641', '1993', ..., '0', '1', '"Biker"'],
       ['60', '641', '1993', ..., '0', '1', '"Biker"'],
       ['60', '641', '1993', ..., '0', '0', '"Biker"']], dtype='<U12')

In [36]:
data.shape

(51462, 10)

In [37]:
xxyy = data[:,1:5]
data = np.delete(data, np.s_[1:5], axis=1)
x = (xxyy[:,0].astype("int") + xxyy[:,2].astype("int") )// 2
y = (xxyy[:,1].astype("int") + xxyy[:,3].astype("int") )// 2
print(data.shape, x.reshape(-1,1).shape, y.reshape(-1,1).shape)
data = np.hstack((data, x.reshape(-1,1), y.reshape(-1,1)))
data

(51462, 6) (51462, 1) (51462, 1)


array([['0', '0', '1', ..., '"Pedestrian"', '840', '1436'],
       ['0', '1', '1', ..., '"Pedestrian"', '840', '1436'],
       ['0', '2', '1', ..., '"Pedestrian"', '840', '1436'],
       ...,
       ['60', '1515', '1', ..., '"Biker"', '669', '2003'],
       ['60', '1516', '1', ..., '"Biker"', '669', '2003'],
       ['60', '1517', '1', ..., '"Biker"', '669', '2003']], dtype='<U21')

In [24]:
data

array([['0', '0', '1', '0', '0', '"Pedestrian"'],
       ['0', '1', '1', '0', '1', '"Pedestrian"'],
       ['0', '2', '1', '0', '1', '"Pedestrian"'],
       ...,
       ['60', '1515', '1', '0', '1', '"Biker"'],
       ['60', '1516', '1', '0', '1', '"Biker"'],
       ['60', '1517', '1', '0', '0', '"Biker"']], dtype='<U12')